# Analítica de datos, Examen parcial 1, 2018-02

## Diego Lamus, A00320776

El dataset que deben analizar contiene los registros del histórico de las personas a las que se les ha otorgado un crédito financiero, indicando quienes pudieron pagarlo sin problemas y quienes no.

La idea es poder predecir, utilizando un modelo de aprendizaje supervisado (KNN) y las variables independientes disponibles qué aspirantes a un crédito tienen mas o menos riesgo de pagar.

Las variables de la hoja de datos son las siguientes:

* ID: El ID del cliente en el banco. Numérico, debe ser positivo y único.
* PudoPagar: Indica si el cliente ha podido pagar el crédito sin problemas (1) o no (0). Variable categórica binaria.
* Estudiante: Indica si el cliente es estudiante (Si) o no lo es (No). Variable categórica binaria.
* Ingresos: Indica la cantidad de dinero que gana el cliente
* Cuenta: Indica el dinero que tiene en la cuenta del banco. Deben ser valores numéricos.

## Entendimiento de los datos

El archivo "creditos_examen.csv" contiene el dataset que tienen que analizar. Se recomienda abrirlo primero en un lector de archivos planos para entender preliminarmente su formato y así poderlo cargar adecuadamente con Python.

Teniendo en cuenta el tipo de problema en cuestión (clasificación o regresión), realice un análisis exploratorio de los datos estableciendo el baseline, verificando la calidad de los datos (tipos de las variables, valores inválidos, excepciones, valores faltantes, etc.), utilizando gráficos para poder entender las distribuciones de los datos e identificar posibles problemas.

### Puntos a desarrollar

* Carguen el archivo en memoria y exploren los datos, tratando de identificar problemas e inconsistencias teniendo en cuenta el diccionario de datos y el contexto del problema. (0.7)
* Obtengan el modelo de referencia (Baseline) y calculen sus métricas de evaluación. (0.3)
* Limpien los datos arreglando los problemas, eliminando columnas, eliminando registros, o ignorándolos, justificando sus decisiones. Una vez resueltos los problemas encontrados, indique la siguiente información: ID del registro, Tipo de ProblemaResolución. (1.2)

## Modelamiento con K-NN

### puntos a desarrollar

* Escogiendo un valor de los parámetros al azar, construyan un modelo K-NN inicial sobre los datos limpios y calcule las métricas de accuracy, error, kappa, precision, recall, especifidad, y F-measure. Analice la calidad del modelo interpretando las métricas (1.0).
* Teniendo en cuenta las métricas más adecuadas para el problema, y siguiendo un protocolo de evaluación adecuado, establezca el mejor valor de los parámetros del modelo. (1.5)
* ¿El valor de los parámetros inicialmente escogido estaba en una situación de overfitting o de underfitting? Explique su respuesta. (0.5)

## Desarrollo de los puntos

### 1. Imports necesarios


In [65]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import KFold, cross_val_score #protocolo de evaluación
from sklearn import neighbors, datasets, metrics
from sklearn import preprocessing, model_selection
import seaborn as sns

### 2. Carga de datos y analisis exploratorio

Cargamos los datos y imprimimos los primeros para verificar la carga

In [28]:
# Cargar datos de csv
df = pd.read_csv('Dataset.txt', sep=";", header=0)
# Ver los primero elementos de los datos para verificar la carga
df.head()

,ID,PudoPagar,Estudiante,Ingresos,Cuenta
0,-1,0,No,3196574,358943
1,-1,0,No,4944040,3857756
2,1,0,No,4436163,3647632
3,2,0,Si,1210613,4085902
4,3,0,No,3176714,5367746


Observamos que la carga de datos se realizo correctamente. luego, imprimimos información del dataframe para ver cuantas entradas existen, que tipo de dato tienen las columnas, y si existen entradas nulas.

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
ID            10000 non-null int64
PudoPagar     10000 non-null int64
Estudiante    9996 non-null object
Ingresos      10000 non-null int64
Cuenta        10000 non-null int64
dtypes: int64(4), object(1)
memory usage: 390.7+ KB


Se observa los siguientes problems:

* Observamos que existen en total 10000 entradas. La columna Estudiante tiene 4 registros nulos, los cuales afectan el modelado con KNN. Por otro lado, se observa que tiene tipo de dato object, lo que significa que es una variable categorica, por lo que el modelo KNN no sabria que hacer con este tipo de dato. Teniendo en cuenta lo anterior se procede a eliminar esta columna.


* El tipo de dato de PudoPagar, el cual seria nuestra variable objetivo es int64. Al modelar con KNN se tomaría como regresión, cuando en realidad es un problema de clasificación. Hay que cambiar el tipo de dato a string.
  

* La columna ID nos brinda poca información ya que no hay una correlación directa entre el ID y las categorias, por lo que también se eliminará esta columna.


Se procede a realizar los cambios

In [30]:
# Eliminar las columnas
df = df.drop(['ID','Estudiante'], axis=1)
# Convertir tipo de dato de PudoPagar a string
df['PudoPagar'] = df['PudoPagar'].astype('str')
# Imprimir de nuevo la información
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
PudoPagar    10000 non-null object
Ingresos     10000 non-null int64
Cuenta       10000 non-null int64
dtypes: int64(2), object(1)
memory usage: 234.5+ KB


En este punto se observa que ya la variable objetivo ya es categorica, y las variables independientes son numericas (Se han resuelto los problemas).  Se procede a separar las variables en dependientes e independientes y a estabelecer la linea base.


In [53]:
# dividir en variables dependiente e independientes
y = df.loc[:,['PudoPagar']]
x = df.loc[:, ['Ingresos', 'Cuenta']] 
# Establecemos la liena base
df[df.PudoPagar=="1"]

,PudoPagar,Ingresos,Cuenta
138,1,1785440,7434991
175,1,1427149,11028998
203,1,2035951,8873471
208,1,4895617,9447996
211,1,2065520,9496953
243,1,1493018,7864282
245,1,3905459,9822384
264,1,3000382,7651766
342,1,4685695,8214100
346,1,4213337,9958246


Vemos que solo 333 de 10000 (3,33%) personas pudiaron pagar. De esta forma, la linea base seria de 96.7%, ya que si el modelo predijera que nadie puede pagar tendria esta accuracy. 

## Modelado con KNN

Para la fase de modelado primero se elige un valor al azar de k y se analizan las metricas.

In [63]:
knn = neighbors.KNeighborsClassifier(n_neighbors=10) # Creamos una instancia de Neighbours Classifier
knn.fit(x, y) # Generamos el modelo con los datos y resultados de entrenamiento
y_pred = knn.predict(x) # realizamos una predicción para el 4

# Metricas
print('accuracy: ' + str(metrics.accuracy_score(y,y_pred)))
print('error:' + str(1-metrics.accuracy_score(y,y_pred)))
print('kappa:' + str(metrics.cohen_kappa_score(y,y_pred)))
print("")
class_report = metrics.classification_report(y, y_pred) 
print(class_report)

accuracy: 0.9739
error:0.026100000000000012
kappa:0.4460282988271931

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      9667
          1       0.74      0.33      0.46       333

avg / total       0.97      0.97      0.97     10000



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


### Análisis de metricas

* accuracy: Observamos que el accuracy es superior a la linea base. De esta forma,  se puede inferir que este modelo desempeña mejor que un modelo aleatorio.
* error: se obtuvo un error del 2.61% que es inferior al error de la linea base (3.33%)
* kappa: eliminando el sesgo del modelo se obtuvo un kappa de 0.44
* precisión: Se tuvo una precisión del 98% para no pago, y del 0.74% para pago sin problemas. Esto significa que de las prediciones que se realizaron sobre pago y no pago el 98% y el 74%, respectivamente correspondian a dicha clase.
* recall: se obtuvo un recall del 100% para no pago, que es relevante en este caso, y del 46% para pago sin problemas. Esto significa que se identificaron todos los que tienen problemas con el pago, pero solo 33% de los que pagan sin problemas.

Dado que tenemos un dataset desbalanceado, en el que el 96.7% de los datos pertenecen a una clase, y el 3.33% pertenecen a otra, la metrica mas adecuada para optimizar el modelo sería kappa, ya que esta metrica trata de ignorar este desbalanceo. 

### Escoger mejor k con holdout

Se procede a establecer diferentes k para evaluar el modelo y se imprime el kappa obtenido para cada k. la evaluación de las metricas se realizara con holdout. Se partira el dataset en 90% entrenamiento, 10% evaluación y se comparará como se comportan las predicciones sobre el conjunto que no hace parte del entrenamiento.



In [66]:
# Particionamiento
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.1, random_state=100)

ks = [1,3,5,7,9,11]
for k in ks: 
    knn = neighbors.KNeighborsClassifier(n_neighbors=k) # Creamos una instancia de Neighbours Classifier
    knn.fit(x_train, y_train) # Generamos el modelo con los datos y resultados de entrenamiento
    y_pred = knn.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test,y_pred)
    print('Kappa for ' + str(k)+ ': '+ str(kappa))

Kappa for 1: 0.3405924523197619
Kappa for 3: 0.3886976996920848
Kappa for 5: 0.37921248669741037
Kappa for 7: 0.36137602179836514
Kappa for 9: 0.37921248669741037
Kappa for 11: 0.3143559674673728


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel

En este caso se obtiene el kappa máximo con k = 3. Este sería el mejor valor para realizar la predicción.